In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
import pandas as pd
import numpy as np
#make compatible with Python 2 and Python 3
# from __future__ import print_function, division, absolute_import 

# Remove warnings
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# import packages

import bs4 as bs
import nltk
from nltk.tokenize import sent_tokenize # tokenizes sentences
import re
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

eng_stopwords = stopwords.words('english')

In [4]:
train = pd.read_csv('../input/nlp-getting-started/train.csv')
test = pd.read_csv('../input/nlp-getting-started/test.csv')

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [6]:
train['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [7]:
np.mean(train['text'].apply(len))

101.03743596479706

In [8]:
train.drop('location',axis=1,inplace=True)
test.drop('location',axis=1,inplace=True)

In [9]:
train[['keyword','target']].groupby(['keyword']).mean().sort_values(by='target',ascending=False).head(15)

,target
keyword,
debris,1.000000
wreckage,1.000000
derailment,1.000000
outbreak,0.975000
oil%20spill,0.973684
typhoon,0.973684
suicide%20bombing,0.969697
suicide%20bomber,0.967742
bombing,0.931034


In [10]:
combine = [train,test]

In [11]:
train['keyword'].fillna(value='Unknown',inplace=True)


In [12]:
test['keyword'].fillna(value='Unknown',inplace=True)

In [13]:
train[['keyword','target']].groupby(['keyword']).mean()

,target
keyword,
Unknown,0.688525
ablaze,0.361111
accident,0.685714
aftershock,0.000000
airplane%20accident,0.857143
...,...
wounded,0.702703
wounds,0.303030
wreck,0.189189


In [14]:
train

,id,keyword,text,target
0,1,Unknown,Our Deeds are the Reason of this #earthquake M...,1
1,4,Unknown,Forest fire near La Ronge Sask. Canada,1
2,5,Unknown,All residents asked to 'shelter in place' are ...,1
3,6,Unknown,"13,000 people receive #wildfires evacuation or...",1
4,7,Unknown,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...
7608,10869,Unknown,Two giant cranes holding a bridge collapse int...,1
7609,10870,Unknown,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,Unknown,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,Unknown,Police investigating after an e-bike collided ...,1


In [15]:
test

,id,keyword,text
0,0,Unknown,Just happened a terrible car crash
1,2,Unknown,"Heard about #earthquake is different cities, s..."
2,3,Unknown,"there is a forest fire at spot pond, geese are..."
3,9,Unknown,Apocalypse lighting. #Spokane #wildfires
4,11,Unknown,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...
3258,10861,Unknown,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,Unknown,Storm in RI worse than last hurricane. My city...
3260,10868,Unknown,Green Line derailment in Chicago http://t.co/U...
3261,10874,Unknown,MEG issues Hazardous Weather Outlook (HWO) htt...


In [16]:
!pip install tweet-preprocessor

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [17]:
import preprocessor as p
proc = []
proc = list(map(lambda text:(p.clean(text)),list(train['text'])))

In [18]:
train['text'] = proc

In [19]:
train

,id,keyword,text,target
0,1,Unknown,Our Deeds are the Reason of this May ALLAH For...,1
1,4,Unknown,Forest fire near La Ronge Sask. Canada,1
2,5,Unknown,All residents asked to 'shelter in place' are ...,1
3,6,Unknown,people receive evacuation orders in California,1
4,7,Unknown,Just got sent this photo from Ruby as smoke fr...,1
...,...,...,...,...
7608,10869,Unknown,Two giant cranes holding a bridge collapse int...,1
7609,10870,Unknown,The out of control wild fires in California ev...,1
7610,10871,Unknown,M1.94 [01:04 UTC]?5km S of Volcano Hawaii.,1
7611,10872,Unknown,Police investigating after an e-bike collided ...,1


In [20]:
proc = []
proc = list(map(lambda text:(p.clean(text)),list(test['text'])))
test['text'] = proc

In [21]:
from gensim.parsing.preprocessing import remove_stopwords
proc = []
proc = list(map(lambda text:(remove_stopwords(text)),list(test['text'])))
test['text'] = proc

In [22]:
from gensim.parsing.preprocessing import remove_stopwords
proc = []
proc = list(map(lambda text:(remove_stopwords(text)),list(train['text'])))
train['text'] = proc

In [23]:
# train_df['text'] = train_df['text'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')


In [24]:
train['text'][1].lower().replace('[^\w\s]',' ').replace('\s\s+', ' ')


'forest near la ronge sask. canada'

In [25]:
x = 'Hi +qlkjhdu81u8y3=-p;[];. Wassup'.lower().replace('[^\w\s]',' ').replace('\s\s+', ' ')
x

'hi +qlkjhdu81u8y3=-p;[];. wassup'

In [26]:
import re

proc = []
proc = list(map(lambda text:re.sub('[^a-zA-Z ]' ,'',text).lower(),list(test['text'])))
test['text'] = proc
proc = []
proc = list(map(lambda text:re.sub('[^a-zA-Z ]' ,'',text).lower(),list(train['text'])))
train['text'] = proc

In [27]:
train

,id,keyword,text,target
0,1,Unknown,our deeds reason may allah forgive,1
1,4,Unknown,forest near la ronge sask canada,1
2,5,Unknown,all residents asked shelter place notified off...,1
3,6,Unknown,people receive evacuation orders california,1
4,7,Unknown,just got sent photo ruby smoke pours school,1
...,...,...,...,...
7608,10869,Unknown,two giant cranes holding bridge collapse nearb...,1
7609,10870,Unknown,the control wild fires california northern sta...,1
7610,10871,Unknown,m utckm s volcano hawaii,1
7611,10872,Unknown,police investigating ebike collided car little...,1


In [28]:
from nltk.stem import WordNetLemmatizer
def lemme(row):
    words = row['text']
    token_tag = pos_tag(words.split())
    # Cell that count nouns (NN) and adjectives (JJ) in review3
    # 7.

    def get_wordnet_pos(treebank_tag):

        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return 'n'


    wnl = WordNetLemmatizer()

    wnl_stems = []
    for pair in token_tag:
        res = wnl.lemmatize(pair[0],pos=get_wordnet_pos(pair[1]))
        wnl_stems.append(res)

    return ' '.join(wnl_stems)

In [29]:
train['text'] = train.apply(lemme,axis=1)
test['text'] = test.apply(lemme,axis=1)

In [30]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

stopwords = stopwords.words('english')

#print(stopwords)

vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = stopwords,   \
                             max_features = 50000,
                             )

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train['text'], train['target'], random_state=0, test_size=.2,stratify=train['target'])

In [32]:
vectorizer.fit(X_train)

CountVectorizer(max_features=50000,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])

In [33]:
print(vectorizer.get_feature_names()[:10])

['aa', 'aaaa', 'aaaaaaallll', 'aaarrrgghhh', 'aal', 'aampb', 'aampw', 'aan', 'aannnnd', 'aar']


In [34]:
train_bag = vectorizer.transform(X_train) #transform to a feature matrix
test_bag = vectorizer.transform(X_test)

In [35]:
pd.DataFrame(train_bag.toarray(), columns=vectorizer.get_feature_names())

,aa,aaaa,aaaaaaallll,aaarrrgghhh,aal,aampb,aampw,aan,aannnnd,aar,...,zombie,zone,zonesthank,zonewolf,zotar,zouma,zrnf,zumiez,zurich,zzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6085,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6086,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6087,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6088,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
# %%time
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import f1_score

# ## Initialize a Random Forest classifier with 50 trees
# # hyperparameter n_estimators always set in instantiation

# forest = RandomForestClassifier(n_estimators = 500,n_jobs=-1)

# # Fit the forest to the training set, using the bag of words as 
# # features and the sentiment labels as the target variable



In [37]:
# forest = forest.fit(train_bag, y_train) # can take 20 seconds to run


In [38]:
# f1_score(forest.predict(train_bag),y_train)

In [39]:
# f1_score(forest.predict(test_bag),y_test)

In [40]:
train_bag

<6090x10202 sparse matrix of type '<class 'numpy.int64'>'
	with 45676 stored elements in Compressed Sparse Row format>

In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

clf_l = LogisticRegression(C=0.45,max_iter=50000,penalty='l2',n_jobs=-1)
# pipe = Pipeline([
#     ('count_vector', CountVectorizer()),  
#     ('clf', LogisticRegression(C=0.25,max_iter=50000,penalty='l2',n_jobs=-1)) 
# ])
clf_l.fit(train_bag,y_train)

LogisticRegression(C=0.45, max_iter=50000, n_jobs=-1)

In [65]:
from sklearn import metrics
predicted = clf_l.predict(test_bag)

In [66]:
predicted

array([0, 0, 0, ..., 1, 0, 1])

In [67]:
print("accuracy :",  metrics.accuracy_score(predicted, y_test))

accuracy : 0.8115561391989494


In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import xgboost as xgb 

clf = xgb.XGBClassifier(n_estimators=100,n_jobs=-1,max_depth=500,learning_rate=0.1)
# pipe = Pipeline([
#     ('count_vector', CountVectorizer()), 
#     ('clf', xgb.XGBClassifier(n_estimators=1000,
#         n_jobs=-1,max_depth=50,learning_rate=0.001)) 
# ])
clf.fit(train_bag,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=500,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [46]:
predicted = clf.predict(test_bag)
print("accuracy :",  metrics.accuracy_score(predicted, y_test))

accuracy : 0.8003939592908733


In [47]:
#Importing All libararies
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import random
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D,Flatten,Dense
from tensorflow.keras import models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split


In [50]:
def create_model():
    
    model = models.Sequential()
    #initialize EfficientNetB0 model with input shape as (224,224,3)
    model.add(layers.Conv2D(filters = 1,kernel_size = 3))
    model.add(layers.MaxPool2D())
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation = 'relu'))
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(2, activation = 'softmax'))
    
    return model

In [51]:
model = create_model()

In [68]:
result = pd.DataFrame(clf_l.predict(vectorizer.transform(test['text'])))

In [69]:
result

,0
0,1
1,1
2,1
3,0
4,1
...,...
3258,1
3259,1
3260,1
3261,1


In [70]:
result=pd.concat([test['id'],result],axis=1)

In [71]:
result

,id,0
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [72]:

result.rename(columns={0:'target'},inplace=True)
result = result.set_index('id')
result

,target
id,
0,1
2,1
3,1
9,0
11,1
...,...
10861,1
10865,1
10868,1


In [73]:
result.to_csv('./result.csv')